In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import shutil

from math import cos, sin, pi
from PIL import Image
from tqdm import tqdm

import tensorflow as tf

from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
# 경로 이동
os.chdir('data')

In [4]:
os.listdir()

['210325_aug_inceptionresnetv2_01_27.h5',
 'aug_rotation_noise_upsidedown.csv',
 'sample_submission.csv',
 'model_210318.h5',
 'train_imgs.zip',
 'baseline_submission.csv',
 '210326_aug_inceptionresnetv2_01_45.h5',
 'baseline_with_augmentation.h5',
 'train_df.csv',
 'train_imgs',
 'model_checkpoint_callback_210318_12.h5',
 'test_imgs',
 'test_imgs.zip',
 'submission_210319_01.csv',
 'submission_210319_02.csv',
 'submission_210325_mocha_01.csv',
 '.ipynb_checkpoints']

In [5]:
# train 데이터 중 10%를 검증 데이터로 사용

# csv 파일 불러오기
data = pd.read_csv('train_df.csv')
submission = pd.read_csv('sample_submission.csv')

# 경로 설정
data_paths = sorted(glob.glob('./train_imgs/*.jpg'))
test_paths = sorted(glob.glob('./test_imgs/*.jpg'))

data['path'] = data_paths

In [6]:
# 데이터 프레임 랜덤하게 분할

# 전체 데이터 중 90%는 학습 데이터 활용
train = data.sample(frac=0.9, random_state=2021)
print('학습 데이터 길이는: ', len(train))

# 전체 데이터 중 10%는 검증 데이터 활용
valid = data.drop(train.index)
print('검증 데이터 길이는: ', len(valid))

학습 데이터 길이는:  3776
검증 데이터 길이는:  419


In [7]:
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

In [8]:
# Augmentation Setting
pixel_shifts = [12]
rotation_angles = [12]
inc_brightness_ratio = 1.2
dec_brightness_ratio = 0.8
noise_ratio = 0.008

In [9]:
# 좌우 반전
def left_right_flip(images, keypoints):
    flipped_keypoints = []
    flipped_images = np.flip(images, axis=1)
    for idx, sample_keypoints in enumerate(keypoints):
        if idx % 2 == 0:
            flipped_keypoints.append(480.-sample_keypoints)
        else:
            flipped_keypoints.append(sample_keypoints)

    # left_right_keypoints_convert
    for i in range(8):
        flipped_keypoints[2+(4*i):4+(4*i)], flipped_keypoints[4+(4*i):6+(
            4*i)] = flipped_keypoints[4+(4*i):6+(4*i)], flipped_keypoints[2+(4*i):4+(4*i)]
    flipped_keypoints[36:38], flipped_keypoints[38:
                                                40] = flipped_keypoints[38:40], flipped_keypoints[36:38]
    flipped_keypoints[44:46], flipped_keypoints[46:
                                                48] = flipped_keypoints[46:48], flipped_keypoints[44:46]

    return flipped_images, flipped_keypoints

In [10]:
# 수직/수평 동시 이동
# forloop에서 shift_x, shift_y 중 하나만 놓으면
# 수직 또는 수평 이동만 따로 시행 가능
def shift_images(images, keypoints):
    # tensor -> numpy
    images = images.numpy()
    shifted_images = []
    shifted_keypoints = []
    for shift in pixel_shifts:
        for (shift_x, shift_y) in [(-shift, -shift), (-shift, shift), (shift, -shift), (shift, shift)]:
            # 이동할 matrix 생성
            M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
            shifted_keypoint = np.array([])
            shifted_x_list = np.array([])
            shifted_y_list = np.array([])
            # 이미지 이동
            shifted_image = cv2.warpAffine(
                images, M, (480, 270), flags=cv2.INTER_CUBIC)
            # 이동한만큼 keypoint 수정
            for idx, point in enumerate(keypoints):
                if idx % 2 == 0:
                    shifted_keypoint = np.append(
                        shifted_keypoint, point+shift_x)
                    shifted_x_list = np.append(shifted_x_list, point+shift_x)
                else:
                    shifted_keypoint = np.append(
                        shifted_keypoint, point+shift_y)
                    shifted_y_list = np.append(shifted_y_list, point+shift_y)
            # 수정된 keypoint가 이미지 사이즈를 벗어나지 않으면 append
            if np.all(0.0 < shifted_x_list) and np.all(shifted_x_list < 480) and np.all(0.0 < shifted_y_list) and np.all(shifted_y_list < 270):
                shifted_images.append(shifted_image.reshape(270, 480, 3))
                shifted_keypoints.append(shifted_keypoint)

    return shifted_images, shifted_keypoints

In [11]:
# 이미지 회전
def rotate_augmentation(images, keypoints):
    # tensor -> numpy
    images = images.numpy()
    rotated_images = []
    rotated_keypoints = []

    for angle in rotation_angles:
        for angle in [angle, -angle]:
            # 회전할 matrix 생성
            M = cv2.getRotationMatrix2D((240, 135), angle, 1.0)
            # cv2_imshow로는 문제없지만 추후 plt.imshow로 사진을 확인할 경우 black screen 생성...
            # 혹시 몰라 matrix를 ndarray로 변환
            M = np.array(M, dtype=np.float32)
            angle_rad = -angle*pi/180
            rotated_image = cv2.warpAffine(images, M, (480, 270))
            rotated_images.append(rotated_image)

            # keypoint를 copy하여 forloop상에서 값이 계속 없데이트 되는 것을 회피
            rotated_keypoint = keypoints.copy()
            rotated_keypoint[0::2] = rotated_keypoint[0::2] - 240
            rotated_keypoint[1::2] = rotated_keypoint[1::2] - 135

            for idx in range(0, len(rotated_keypoint), 2):
                rotated_keypoint[idx] = rotated_keypoint[idx] * \
                    cos(angle_rad)-rotated_keypoint[idx+1]*sin(angle_rad)
                rotated_keypoint[idx+1] = rotated_keypoint[idx] * \
                    sin(angle_rad)+rotated_keypoint[idx+1]*cos(angle_rad)

            rotated_keypoint[0::2] = rotated_keypoint[0::2] + 240
            rotated_keypoint[1::2] = rotated_keypoint[1::2] + 135
            rotated_keypoints.append(rotated_keypoint)

    return rotated_images, rotated_keypoints

In [12]:
# 이미지 해상도 조절
def alter_brightness(images):
    altered_brightness_images = []
    inc_brightness_images = np.clip(images*inc_brightness_ratio, 0.0, 1.0)
    dec_brightness_images = np.clip(images*dec_brightness_ratio, 0.0, 1.0)
    altered_brightness_images.append(inc_brightness_images)
    altered_brightness_images.append(dec_brightness_images)
    return altered_brightness_images

In [13]:
# Random 노이즈 추가
def add_noise(images):
    images = images.numpy()
    noise = noise_ratio * np.random.randn(270, 480, 3)
    noise = noise.astype(np.float32)
    # 생성한 noise를 원본에 add
    noisy_image = cv2.add(images, noise)
    return noisy_image

In [14]:
def trainGenerator():
    # 원본 이미지 resize
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])  # path(경로)를 통해 이미지 읽기
        # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])   # 이미지 resize
        img = img/255                            # 이미지 rescaling
        target = train.iloc[:, 1:49].iloc[i, :]  # keypoint 뽑아주기
        # image size를 1920x1080 -> 480x270으로 바꿔줬으므로 keypoint도 변경
        target = target/4
        
        yield (img, target)

    # horizontal flip
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]
        target = target/4
        
        img, target = left_right_flip(img, target)        
        
        yield (img, target)

    # Horizontal & Vertical shift
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]
        target = target/4
        
        img_list, target_list = shift_images(img, target)
        
        for shifted_img, shifted_target in zip(img_list, target_list):
            yield (shifted_img, shifted_target)

    # Rotation
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]
        target = target/4
        
        img_list, target_list = rotate_augmentation(img, target)
        
        for rotated_img, rotated_target in zip(img_list, target_list):
            yield (rotated_img, rotated_target)

    # Alter_Brightness
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]
        target = target/4
        
        img_list = alter_brightness(img)
        
        for altered_brightness_images in img_list:
            yield (altered_brightness_images, target)

    # Adding_Noise
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]
        target = target/4
        
        noisy_img = add_noise(img)
        
        yield (noisy_img, target)

In [15]:
def validGenerator():
    # 원본 이미지 resize
    for i in range(len(valid)):
        img = tf.io.read_file(valid['path'][i])  # path(경로)를 통해 이미지 읽기
        # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])   # 이미지 resize
        img = img/255                            # 이미지 rescaling
        target = valid.iloc[:, 1:49].iloc[i, :]  # keypoint 뽑아주기
        # image size를 1920x1080 -> 480x270으로 바꿔줬으므로 keypoint도 변경
        target = target/4

        yield (img, target)

In [16]:
batch_size = 256

In [17]:
train_dataset = tf.data.Dataset.from_generator(
    trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([270, 480, 3]), tf.TensorShape([48])))
train_dataset = train_dataset.batch(batch_size).prefetch(AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(
    validGenerator, (tf.float32, tf.float32), (tf.TensorShape([270, 480, 3]), tf.TensorShape([48])))
valid_dataset = valid_dataset.batch(batch_size).prefetch(AUTOTUNE)

In [18]:
# Callback 설정
earlystopping = EarlyStopping(patience=10)

# learning_rate_reduction = ReduceLROnPlateau(
#     monitor="val_loss",
#     patience=3,
#     factor=0.01,
#     min_lr=0,
#     verbose=1
# )


def my_learning_rate(epoch, lrate):
    return lrate*(1/(1+0.01*3))


lrs = LearningRateScheduler(my_learning_rate)

model_check = ModelCheckpoint(
    filepath="./210326_aug_mobilenetv3small_01_{epoch:04d}.h5",  # 모델 파일 경로
    monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
    save_best_only=True
)

callbacks = [earlystopping, lrs, model_check]

In [19]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():

    base_model = MobileNetV3Small(
        input_shape=(270, 480, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg',
        dropout_rate=0.8
    )

    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)
    predictions = Dense(48)(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=RMSprop(
            learning_rate=0.1,
            momentum=0.9
        ),
        loss='mean_squared_error',
        metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 270, 480, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 270, 480, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 135, 240, 16) 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 135, 240, 16) 64          Conv[0][0]                       
______________________________________________________________________________________________

In [ ]:
history = model.fit(
    train_dataset,
    epochs=10000,
    validation_data=valid_dataset,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10000
INFO:tensorflow:batch_all_reduce: 146 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 146 all-reduces with algorithm = nccl, num_packs = 1
163/163 [==============================] - 697s 4s/step - loss: 54701.7998 - accuracy: 0.0581 - val_loss: inf - val_accuracy: 0.0000e+00
Epoch 2/10000
163/163 [==============================] - 658s 4s/step - loss: 2219.8959 - accuracy: 0.1385 - val_loss: inf - val_accuracy: 0.0072
Epoch 3/10000
163/163 [==============================] - 648s 4s/step - loss: 1826.1892 - accuracy: 0.1413 - val_loss: 220082663782608333388740167351664640.0000 - val_accuracy: 0.1575
Epoch 4/10000
105/163 [==================>...........] - ETA: 3:34 - loss: 1724.9906 - accuracy: 0.1403